
# Task 1 Clean and Upload Data


In [1]:
import os 
import io
from zipfile import ZipFile
import pandas as pd
import csv
from google.cloud import bigquery
from pandas_gbq import to_gbq
#from google.oauth2 import service_account

#### Connecting to GBQ

### Configure Goggle Big Query

In [2]:
gbq_project_ID = "wedge_project_np"
gbq_dataset_ID = "wedge"

client = bigquery.Client(project = "wedge_project_np")
credentials = client._credentials

#### Opening Zip Files

In [3]:
data_directory = "Data/SmallZip/"
zip_files = os.listdir("Data/SmallZip")

In [ ]:
### How is this different compared to below. I didn't extract below and need to??

#with ZipFile(data_directory+file) as my_zip:
    #print(my_zip.namelist())
    #for zipped_file in my_zip.namelist():
        #x = my_zip.extract(zipped_file)

### Iterate Over Zip Files and Save to Pandas Dataframe

In [32]:
# Iterate over all zip files in the directory
for current_zf in zip_files:
    with ZipFile(data_directory + current_zf, 'r') as zf:
        zipped_files = zf.namelist()

        # Iterate over each file in the zip
        for file_name in zipped_files:
            # Assuming the file is a CSV, we will load it into a DataFrame
            with zf.open(file_name) as file:
                # Read file as CSV
                try:
                    df = pd.read_csv(file, quotechar='"') 
                    print(f"Loaded {file_name} into DataFrame.")
                    print(df.head())  # Check the first few rows of the DataFrame
                except Exception as e:
                    print(f"Error reading {file_name}: {e}")
                    
        print("\n")

Loaded transArchive_201001_201003_small.csv into DataFrame.
              datetime  register_no  emp_no  trans_no            upc  \
0  2010-01-01 09:04:09            5      17         2  0005385200400   
1  2010-01-01 09:04:12            5      17         2  0020631400000   
2  2010-01-01 09:04:15            5      17         2  0002433551303   
3  2010-01-01 09:04:18            5      17         2  0004850000139   
4  2010-01-01 09:04:21            5      17         2  0004850000139   

                      description trans_type trans_subtype trans_status  \
0           Medium Salsa 16oz GMG          I                              
1   ChickenBreastSkinlessBoneless          I                              
2       Taco Seasoning 1.4oz Bear          I                              
3  Orange Juice/Some Pulp 64oz Tr          I                              
4  Orange Juice/Some Pulp 64oz Tr          I                              

   department  ...  batchHeaderID  local  organic  displ

In [ ]:
"""#current_zf = zip_files[0]
for current_zf in zip_files :
# Open the current zipfile
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        # Iteraate over each file inside the current zip file
        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")

            for idx, line in enumerate(input_file) :
                print(line)
                if idx > 4 :
                    break
            input_file.close()
        print("\n")"""


## Sniffing out the Delimiter 

In [33]:
delimiters = dict() 

# Start by reading in all the files again.

#current_zf = zip_files[0]
for current_zf in zip_files :
# Open the current zf
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        # Iteraate over each file inside the current zip file
        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["For",
                           file_name,
                           "the delimiter is",
                           dialect.delimiter
                           ]))

            input_file.close() # tidy up

For transArchive_201001_201003_small.csv the delimiter is ,
For transArchive_201004_201006_small.csv the delimiter is ,
For transArchive_201007_201009_small.csv the delimiter is ,
For transArchive_201010_201012_small.csv the delimiter is ,
For transArchive_201101_201103_small.csv the delimiter is ,
For transArchive_201104_small.csv the delimiter is ,
For transArchive_201105_small.csv the delimiter is ,
For transArchive_201106_small.csv the delimiter is ,
For transArchive_201107_201109_small.csv the delimiter is ,
For transArchive_201110_201112_small.csv the delimiter is ,
For transArchive_201201_201203_inactive_small.csv the delimiter is ;
For transArchive_201201_201203_small.csv the delimiter is ,
For transArchive_201204_201206_inactive_small.csv the delimiter is ;
For transArchive_201204_201206_small.csv the delimiter is ,
For transArchive_201207_201209_inactive_small.csv the delimiter is ;
For transArchive_201207_201209_small.csv the delimiter is ,
For transArchive_201210_201212_ina

## Prinint out First Line

In [35]:
for this_zf in zip_files :
    with ZipFile(data_directory + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            this_delimiter = delimiters[file_name]
            
            for line in input_file :
                print(line.strip().split(this_delimiter))
                break

            input_file.close() # tidy up

['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicable"', '"discount"', '"memDiscount"', '"discountable"', '"discounttype"', '"voided"', '"percentDiscount"', '"ItemQtty"', '"volDiscType"', '"volume"', '"VolSpecial"', '"mixMatch"', '"matched"', '"memType"', '"staff"', '"numflag"', '"itemstatus"', '"tenderstatus"', '"charflag"', '"varflag"', '"batchHeaderID"', '"local"', '"organic"', '"display"', '"receipt"', '"card_no"', '"store"', '"branch"', '"match_id"', '"trans_id"']
['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicabl

### Working Checking for Headers Row

In [37]:
headers = dict()

def is_header_row(first_row, second_row):

    # Check if most elements in the first row contain non-numeric characters
    if all(any(c.isalpha() for c in value) for value in first_row):
        return True
    
    # Optionally: Check if types of first and second rows differ
    if set(map(type, first_row)) != set(map(type, second_row)):
        return True
    
    return False


#current_zf = zip_files[0]
for current_zf in zip_files :
# Open the current zf
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        # Loop through each file in the zip
        for file_name in zipped_files:
            with zf.open(file_name, 'r') as input_file:
                input_file = io.TextIOWrapper(input_file, encoding="utf-8")

                this_delimiter = delimiters.get(file_name, ',')  # Use delimiter or default to ','

                # Read the first two lines
                first_line = input_file.readline().strip().split(this_delimiter)
                second_line = input_file.readline().strip().split(this_delimiter)

                # Check for header presence using improved logic
                has_header = is_header_row(first_line, second_line)
                print(f"File '{file_name}' has header: {has_header}")

                # Print first two lines for verification
                print("First line:", first_line)
                print("Second line:", second_line)


                headers[file_name] = has_header


                input_file.close()  # Close the file properly
    

File 'transArchive_201001_201003_small.csv' has header: True
First line: ['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicable"', '"discount"', '"memDiscount"', '"discountable"', '"discounttype"', '"voided"', '"percentDiscount"', '"ItemQtty"', '"volDiscType"', '"volume"', '"VolSpecial"', '"mixMatch"', '"matched"', '"memType"', '"staff"', '"numflag"', '"itemstatus"', '"tenderstatus"', '"charflag"', '"varflag"', '"batchHeaderID"', '"local"', '"organic"', '"display"', '"receipt"', '"card_no"', '"store"', '"branch"', '"match_id"', '"trans_id"']
Second line: ['"2010-01-01 09:04:09"', '"5"', '"17"', '"2"', '"0005385200400"', '"Medium Salsa 16oz GMG"', '"I"', '" "', '" "', '"1"', '"1"', '"0"', '"2.6480"', '"2.9900"', '"2.9900"', '"3.9900"', '"0.0000"', '"0"', 

In [27]:
# Check if the dictionary is empty
if not headers:
    print("The 'headers' dictionary is empty.")
else:
    print("The 'headers' dictionary contains:")
    print(headers)

The 'headers' dictionary contains:
{'transArchive_201001_201003_small.csv': True, 'transArchive_201004_201006_small.csv': True, 'transArchive_201007_201009_small.csv': True, 'transArchive_201010_201012_small.csv': True, 'transArchive_201101_201103_small.csv': True, 'transArchive_201104_small.csv': True, 'transArchive_201105_small.csv': True, 'transArchive_201106_small.csv': True, 'transArchive_201107_201109_small.csv': True, 'transArchive_201110_201112_small.csv': True, 'transArchive_201201_201203_inactive_small.csv': True, 'transArchive_201201_201203_small.csv': True, 'transArchive_201204_201206_inactive_small.csv': True, 'transArchive_201204_201206_small.csv': True, 'transArchive_201207_201209_inactive_small.csv': True, 'transArchive_201207_201209_small.csv': True, 'transArchive_201210_201212_inactive_small.csv': True, 'transArchive_201210_201212_small.csv': True, 'transArchive_201301_201303_inactive_small.csv': True, 'transArchive_201301_201303_small.csv': True, 'transArchive_201304

#### Cleaning and Processing
-- same loop as above but with processing.

In [63]:
def missing_values(df, schema):
    for field in schema:
        field_name = field.name
        field_type = field.field_type
        
        # Handle missing values based on field type
        if field_type == "FLOAT":
            df[field_name] = df[field_name].fillna(0.0)
        elif field_type == "STRING":
            df[field_name] = df[field_name].fillna('')
        elif field_type == "TIMESTAMP":
            df[field_name] = pd.to_datetime(df[field_name], errors='coerce')
        elif field_type == "BOOLEAN":
            df[field_name] = df[field_name].fillna(False)
    
    return df

In [ ]:
processed_files_directory = "data/processed_files/"  # Path to store processed files

# Create the processed files directory if it doesn't exist
if not os.path.exists(processed_files_directory):
    os.makedirs(processed_files_directory)

# Iterate over your zip files
for current_zf in zip_files:
    with ZipFile(data_directory + current_zf, 'r') as zf:
        zipped_files = zf.namelist()

        # Iterate over each file in the zip
        for file_name in zipped_files:
            with zf.open(file_name) as working_file:
                try:
                    # Read in the file into a pandas DataFrame
                    df = pd.read_csv(working_file)
                    print(f"Loaded {file_name} into DataFrame.")

                    # Strip any whitespace from column names
                    df.columns = df.columns.str.strip()

                    # Example of filling missing values and cleaning
                   # df['register_no'] = df['register_no'].fillna(0) 
                   # df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
                    #df['quantity'] = df['quantity'].fillna(0)  
                    df = missing_values(df, schema)

                    # Save the processed DataFrame to the processed files directory
                    processed_filename = os.path.join(processed_files_directory, f"processed_{file_name}")
                    df.to_csv(processed_filename, index=False)

                    print(f"Processed and saved {file_name} to {processed_filename}.")

                except Exception as e:
                    print(f"Error processing {file_name}: {e}")

        print("\n")


In [ ]:
processed_files = os.listdir("data/processed_files")

if not os.path.exists(processed_files_directory):
    os.makedirs(processed_files_directory)

for current_zf in zip_files:
    with ZipFile(data_directory + current_zf, 'r') as zf:
        zipped_files = zf.namelist()

        # Iterate over each file in the zip
        for file_name in zipped_files:
            with zf.open(file_name) as working_file:
                # Process one file at a time
                try:
                    # Read in the file into a pandas DataFrame
                    df = pd.read_csv(working_file)
                    print(f"Loaded {file_name} into DataFrame.")

                    df.columns = df.columns.str.strip()
                    
                    # Use the clean dataframe function to handle missing values
                    df['register_no'].fillna(0, inplace=True)
                    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
                    df['quantity'] = df['quantity'].fillna(0)

                    #df = missing_values(df, schema)
                    
                    # Example data type conversions (based on schema matching)
                    #df['register_no'] = df['register_no'].astype(float)
                    #df['quantity'] = df['quantity'].astype(float)
                    #df['upc'] = df['upc'].astype(str)
                    
                    # 3. Optional: Validate and clean data further
                    #df = df.dropna(subset=['datetime'])  # Drop rows with missing critical fields
                    
                    # 4. Do something with the processed DataFrame (e.g., save, upload, etc.)
                    # Example: Save to a cleaned CSV or upload to BigQuery
                    
                    processed_filename = os.path.join(processed_files, f"processed_{file_name}")
                    df.to_csv(processed_filename, index=False)

                    # (Alternatively, you could upload to BigQuery at this point)
                    print(f"Processed and saved {file_name}.")
                    
                except Exception as e:
                    print(f"Error processing {file_name}: {e}")
                  
        print("\n")

In [ ]:
print(df.columns.tolist()) 

Loading to GBQ

In [ ]:
full_path= data_directory + current_zf
if os.path.exists(full_path):
    print("File found")

    with open(data_directory + current_zf, 'rb') as source_file:
        job = client.load_table_from_file(
            source_file,
            f"{gbq_dataset_id}.{current_zf}",
            job_config=job_config,
        )
else:
    print("File not found")

In [ ]:
job.result()

### Create table names for tables in GBQ

In [ ]:

for file in data_directory:
    tab, _ = file.replace("_clean.csv", "")
    table_full_name = ".".join([gbq_project_id, gbq_dataset_id, tab])

    if not table_exists(client, table_full_name):
        table_ref = client.create_table(table=table_full_name)
    else:
        table_ref = client.get_table(table_full_name)

    table = client.get_table(table_ref)
    print("Table {} contains {} columns".format(table_ref.table_id, len(table.schema)))

In [ ]:
tab, _ = file.replace("_clean.csv", "")
table_full_name = ".".join([gbq_project_id, gbq_dataset_id, tab])

if not table_exists(client, table_full_name):
    table_ref = client.create_table(table=table_full_name)
else:
    table_ref = client.get_table(table_full_name)
